# CS 329 E Data Visualization Homework 5

## National Parks and Biodiversity 

### Submitted by : Michel Gonzalez (11am)

In our last homework, we'll combine some of the concepts from the 
recent lab (geo data) with merging data and create an interactive 
dashboard. 


In [1]:
!pip install geopandas

In [2]:
# importing packages

# data wrangling
import pandas as pd
import numpy as np

# plotting
import altair as alt

# working with geographic data
import json
import geopandas as gpd
from vega_datasets import data

## Loading in a shape file
This zip file contains a _shape_ file.  This is one of the types of geometry data supported by geopandas ([doc](https://geopandas.org/en/stable/docs/user_guide/io.html?highlight=shape%20files#reading-spatial-data))

In [3]:
# importing shapefile - We can use a zip file that contains a shape file by reading it from a URL
gdf = gpd.read_file('http://cs.utexas.edu/~chaney/nps_boundary.zip')

In [4]:
gdf.sample(5)

,UNIT_CODE,GIS_Notes,UNIT_NAME,DATE_EDIT,STATE,REGION,GNIS_ID,UNIT_TYPE,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area,Unified_Re,Old_Region,geometry
206,LOSA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Lower Saint Croix National Scenic Riverway,2016-05-10,WI,MW,2738286,National Scenic Riverway,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Lower Saint Croix,1.080329,0.005243,3,MW,"MULTIPOLYGON (((-92.67075 45.38886, -92.67083 ..."
224,FOSM,Lands - http://landsnet.nps.gov/tractsnet/docu...,Fort Smith National Historic Site,2017-03-14,AR,MW,56686,National Historic Site,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Fort Smith,0.057419,0.000029,"4, 6",MW,"MULTIPOLYGON (((-94.42893 35.38744, -94.42914 ..."
148,MABI,Lands - http://landsnet.nps.gov/tractsnet/docu...,Marsh-Billings-Rockefeller National Historical...,2011-09-21,VT,NE,1888942,National Historical Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Marsh - Billings - Rockefeller,0.131633,0.000291,1,NE,"MULTIPOLYGON (((-72.51176 43.63500, -72.51150 ..."
155,SAPU,Lands - http://landsnet.nps.gov/tractsnet/docu...,Salinas Pueblo Missions National Monument,2013-11-27,NM,IM,936877,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Salinas Pueblo Missions,0.186398,0.000425,7,IM,"MULTIPOLYGON (((-106.07613 34.26372, -106.0759..."
395,MISP,Lands - http://landsnet.nps.gov/tractsnet/docu...,Mill Springs Battlefield National Monument,2020-09-23,KY,SE,498296,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Mill Springs Battlefield,0.215502,0.000603,1,SE,"MULTIPOLYGON (((-84.77821 36.91738, -84.77822 ..."


## Q1 - Overlay the National Parks and Monuments on a county map of the USA
Using the geopandas data frame `gdf`, filter out just the national parks and monuments and overlay
them on a county map of the USA.  Use the Vega data set to map the counties, using [this page](https://altair-viz.github.io/gallery/choropleth.html) as a reference.  Create a tool tip that shows the park name (`UNIT_NAME`) and region (`REGION`) of the park, title your chart, and encode the color with the Region of the park using the `dark2` colorscheme. See [this page](https://altair-viz.github.io/user_guide/customization.html#customizing-colors) for a reference on changing the color scheme. 

This would be an appropriate visualization if our user wanted to understand the landsize of the parks and how the regions were labeled. 

In [5]:
# your code here

p_m = gdf[((gdf['UNIT_TYPE'] == 'National Monument') | (gdf['UNIT_TYPE'] == 'National Park')) & ((gdf['REGION'] == 'IM') | (gdf['REGION'] == 'PW'))]

p_m.geometry = p_m.geometry.simplify(.01) #speeds up the process by a lot & keeps the general shape

counties = alt.topo_feature(data.us_10m.url, 'counties')

basemap = alt.Chart(counties).mark_geoshape(
    
    opacity= 0.25,
    
    fill="lightgray", 
    
    stroke="darkgray",
    
).project(
    
    type = 'albersUsa',
    
).properties(
    
    width=1500, 
        
    height=500,
)

areas = alt.Chart(p_m).mark_geoshape().encode(
    
    color=alt.Color('REGION:O', scale = alt.Scale(scheme='dark2')),

    tooltip=['UNIT_NAME', 'REGION'],

).properties(
    
    width=1500, 
        
    height=500,

    title = 'National Parks and Monuments in the USA',
)

basemap + areas

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


alt.LayerChart(...)

## Loading in a csv a species data

In [6]:
df_species = pd.read_csv('http://cs.utexas.edu/~chaney/numspecies.csv')
df_species = df_species.fillna( value = 0 ) # Fill the missing species counts with zeros for visualization
df_species.sample(5)

,UNIT_CODE,All Types,Amphibian,Bird,Fish,Mammal,Reptile,Vascular Plant,Fungi,Insect,Invertebrate,Spider/Scorpion,Nonvascular Plant,Algae,Crab/Lobster/Shrimp,Slug/Snail
7,CANY,1223,7.0,220,33.0,76,22.0,865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,LAVO,1797,17.0,245,20.0,100,22.0,1046,51.0,95.0,18.0,0.0,160.0,2.0,15.0,6.0
40,NOCA,3363,12.0,226,33.0,79,10.0,1478,539.0,559.0,0.0,42.0,385.0,0.0,0.0,0.0
30,ISRO,1397,13.0,261,63.0,26,5.0,993,0.0,0.0,21.0,0.0,1.0,0.0,14.0,0.0
34,KOVA,1025,1.0,125,26.0,38,0.0,476,333.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0


## Q2 - Change data to Long format 
Remember [long form vs wide form formats](https://altair-viz.github.io/user_guide/data.html#long-form-vs-wide-form-data)? It turns out our species data is wide form.  Convert it to long form and save the new dataframe to `df_species_long`. Name your new columns `Species Type` and `Species Count` so they look pretty when we plot them later.

In [7]:
# your code here

df_species_long = df_species.melt('UNIT_CODE', var_name='Species Type', value_name='Species Count')

In [8]:
# sniff check
df_species_long.sample(5)

,UNIT_CODE,Species Type,Species Count
535,JOTR,Invertebrate,10.0
434,PEFO,Fungi,1.0
609,THRO,Spider/Scorpion,2.0
717,ROMO,Algae,150.0
388,WRST,Vascular Plant,1357.0


## Q3 - Get lat/long and area from park shape
We have the shape file for the national parks, which contains the polygon for the park boundary, but we decide for the visualization we want to encode the location with a circle whose size is constant.  We also are interested in comparing the area of the park with the species diversity to discover trends. To accomplish these tasks we need to calculate the centroid of the park to get a single lat/long and calculate the area of the polygon in acres (a convenient unit for the USA). 

To do math in geopandas you have to translate between "EPSG:4269" and "EPSG:5070" (USA Albers projection)

Add the `Acres` column to the `gdf` geopandas dataframe, and replace the `geometry` column with the centroid. Use the [intro to geopandas](https://geopandas.org/en/stable/getting_started/introduction.html) documentation to help with the syntax.

Check that everything looks OK by plotting the lat/long of the parks on top of the county map as before. Include a tooltip that shows the region, name of park, and Acres. Use the `dark2` color scheme.

I'm helping you out with the coordinate reference systems here; note that there are many many map reference coordinates and we can see the reference for a given geopandas dataframe with the `crs` property. 

In [9]:
# look at the coordinate reference of the data - this needs to be translated to do math, but then put back
gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [10]:
gdf = gdf.to_crs( 'EPSG:5070') 

In [11]:
gdf.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
# Find the area in acres

# The units of the area is in meters squared & there are 4046.86 meters squared in one acre

gdf['Acres'] = (gdf.area)/4046.86 # your code here to calculate acrage 

In [13]:
# sniff check
# The value from Rocky Mountain National Park webpage is 265,807 acres and our estimate is off probably due to the 
# projection we used was not locally optimal, or maybe the shape file wasn't exact. If we were a GIS class
# this would probably bother us, but for now we'll call this close enough (your number should be about 0.5% off)
gdf[gdf['UNIT_CODE'] == 'ROMO']['Acres']

415    267041.413525
Name: Acres, dtype: float64

In [14]:
gdf['geometry'] = gdf.centroid # your code here to convert the polygons data to point data 

In [15]:
gdf = gdf.to_crs( 'EPSG:4269') # This takes us back to lat/long space
gdf.sample(5)

,UNIT_CODE,GIS_Notes,UNIT_NAME,DATE_EDIT,STATE,REGION,GNIS_ID,UNIT_TYPE,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area,Unified_Re,Old_Region,geometry,Acres
298,BELA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Bering Land Bridge National Preserve,2019-03-21,AK,AK,1416037,National Preserve,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Bering Land Bridge,15.807596,2.222259e+00,11,AK,POINT (-164.39687 65.95278),2.785107e+06
72,FOMA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Fort Matanzas National Monument,2009-10-26,FL,SE,307963,National Monument,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Fort Matanzas,0.109911,1.033384e-04,2,SE,POINT (-81.23568 29.71215),2.738934e+02
140,JOFL,Lands - http://landsnet.nps.gov/tractsnet/docu...,Johnstown Flood National Memorial,2012-08-28,PA,NE,1200751,National Memorial,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Johnstown Flood,0.066742,7.707602e-05,1,NE,POINT (-78.77425 40.34587),1.796832e+02
229,CAVE,Lands - http://landsnet.nps.gov/tractsnet/docu...,Carlsbad Caverns National Park,2017-06-22,NM,IM,904780,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Carlsbad Caverns,0.985364,1.810148e-02,7,IM,POINT (-104.55304 32.14089),4.679721e+04
124,VIVE,Lands - http://landsnet.nps.gov/tractsnet/docu...,Vietnam Veterans Memorial,2011-11-03,DC,NC,531832,National Memorial,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Vietnam Veterans,0.004339,9.180628e-07,1,NC,POINT (-77.04766 38.89096),2.184968e+00


In [16]:
# your map code here

counties = alt.topo_feature(data.us_10m.url, 'counties')

basemap = alt.Chart(counties).mark_geoshape(
    
    opacity= 0.25,
    
    fill="lightgray", 
    
    stroke="darkgray",
    
).project(
    
    type = 'albersUsa'
    
).properties(
    
    width=1500, 
        
    height=500,
)

points = alt.Chart(gdf[(gdf['UNIT_TYPE'] == 'National Monument') | (gdf['UNIT_TYPE'] == 'National Park')]).mark_circle(size = 70).encode(
    
    longitude='geometry.coordinates[0]:Q', 

    latitude='geometry.coordinates[1]:Q',

    tooltip = ['UNIT_NAME', 'REGION', 'Acres'],

    color=alt.Color('REGION:O', scale = alt.Scale(scheme='dark2')),

).properties(width=600, height=600, title = 'National Parks and Monuments in the USA')

basemap + points

alt.LayerChart(...)

## Q4 - Merge Geo Data and and Species Data
We've been using the `UNIT_NAME` for the park name in the plots, and we want to use that same name, and the acerage
when plotting the species data.  Add these two columns to our `df_species_long` dataframe using a inner join, renaming the `UNIT_NAME` to `Park Name`, and `REGION` to `Region` for pretty 
visualization. The key for the merge is `UNIT_CODE`. Save this new dataframe as `df_species_final`

In [17]:
# your code here

new_gdf = gdf.copy(deep = True)

new_gdf.rename(columns = {'UNIT_NAME':'Park Name', 'REGION':'Region'}, inplace = True)

new_gdf = new_gdf[new_gdf['UNIT_TYPE'] == 'National Park']

df_species_final = pd.merge(df_species_long, new_gdf, on = 'UNIT_CODE')

df_species_final = df_species_final[['UNIT_CODE', 'Species Type', 'Species Count', 'Park Name', 'Acres', 'Region']]

df_species_final

new_gdf

,UNIT_CODE,GIS_Notes,Park Name,DATE_EDIT,STATE,Region,GNIS_ID,UNIT_TYPE,CREATED_BY,METADATA,PARKNAME,Shape_Leng,Shape_Area,Unified_Re,Old_Region,geometry,Acres
35,DRTO,Lands - http://landsnet.nps.gov/tractsnet/docu...,Dry Tortugas National Park,2007-11-07,FL,SE,307632,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Dry Tortugas,0.601299,0.023667,2,SE,POINT (-82.87179 24.64884),6.557793e+04
38,BRCA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Bryce Canyon National Park,2008-03-27,UT,IM,1450197,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Bryce Canyon,1.006552,0.014856,7,IM,POINT (-112.18279 37.58385),3.598693e+04
41,YELL,Lands - http://landsnet.nps.gov/tractsnet/docu...,Yellowstone National Park,2008-04-23,WY,IM,1609331,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Yellowstone,5.215542,1.008832,"5, 7",IM,POINT (-110.54725 44.59558),2.199453e+06
51,SEQU,Lands - http://landsnet.nps.gov/tractsnet/docu...,Sequoia National Park,2008-11-26,CA,PW,266000,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Sequoia,2.608109,0.165609,10,PW,POINT (-118.57521 36.50769),4.068057e+05
57,MACA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Mammoth Cave National Park,2009-03-27,KY,SE,507318,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Mammoth Cave,1.177721,0.021232,1,SE,POINT (-86.13088 37.19758),5.169496e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,DEVA,Lands - http://landsnet.nps.gov/tractsnet/docu...,Death Valley National Park,2020-01-22,CA,PW,857501,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Death Valley,7.343349,1.397857,"8, 1",PW,POINT (-117.12611 36.47659),3.434911e+06
381,HOSP,Lands - http://landsnet.nps.gov/tractsnet/docu...,Hot Springs National Park,2020-06-15,AR,MW,56796,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Hot Springs,0.498187,0.002168,4,MW,POINT (-93.06335 34.52411),5.457056e+03
415,ROMO,Lands - http://landsnet.nps.gov/tractsnet/docu...,Rocky Mountain National Park,2021-02-22,CO,IM,204948,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Rocky Mountain,2.338432,0.114565,7,IM,POINT (-105.69728 40.35535),2.670414e+05
419,MEVE,Lands - http://landsnet.nps.gov/tractsnet/docu...,Mesa Verde National Park,2021-03-08,CO,IM,179042,National Park,Lands,https://irma.nps.gov/DataStore/Reference/Profi...,Mesa Verde,0.819376,0.022063,7,IM,POINT (-108.46242 37.23904),5.368831e+04


## Q5 - Compare the Acres to Species Count

On a log/log scale compare the total species count (only the rows where "Species Type" is "All Types") with the park size. Encode the region using the same color scale as above. Add a tool tip to see the park name, Species Count, and Acres.

Add a `selection_multi` that will highlight the selected data by turning unselected data light gray.

In [18]:
# your code here

park_select = alt.selection_multi(fields = ['Park Name'])

scatter = alt.Chart(df_species_final[df_species_final['Species Type'] == 'All Types']).mark_circle(size = 70).encode(
    
    x = alt.X('Acres:Q',
              scale = alt.Scale(type = 'log')
              ),

    y = alt.Y('Species Count:Q',
              scale = alt.Scale(type = 'log')
              ),

    color = alt.condition(park_select,
                      alt.Color('Region:O', scale = alt.Scale(scheme='dark2')),
                      alt.value('lightgray')
                      ),
    tooltip = ['Park Name', 'Species Count', 'Acres']

).add_selection(
    
    park_select
    
).properties( width = 400, height = 300, title = 'Park Size vs Specific Counts (log/log Scale)')

scatter

alt.Chart(...)

## Q6 - Create a rug plot of Species Diversity
Look at how the different Species categores (all Species types that are NOT 'All Types' are distributed
across all the parks.  

Use the same color encoding for the Region as the prior plots. 

Add a `selection_multi` that will highlight the selected data for a park by turning unselected data light gray. 
Also, make the tick a little bigger when it is selected to help find the park in the other categories.

In [19]:
# your code here

park_select = alt.selection_multi(fields = ['Park Name'])

plot = alt.Chart(df_species_final[df_species_final['Species Type'] != 'All Types']).mark_tick(thickness = 3).encode(
    
    x = alt.X('Species Count:Q'),

    y = alt.Y('Species Type:O', title = ''),

    color = alt.condition(park_select,
                          alt.Color('Region:O', scale = alt.Scale(scheme='dark2')),
                          alt.value('lightgray'),
                          ),

    size = alt.condition(park_select,
                         alt.value(20),
                         alt.value(15)
                         ),

    opacity = alt.condition(park_select,
                            alt.value(1),
                            alt.value(0.5),
                            ),
    
    tooltip = ['Park Name', 'Species Count', 'Species Type', 'Region'],

).add_selection(
    
    park_select

).properties(width = 1000, height = 400, title = 'Species Count by Type')

plot


alt.Chart(...)

## Q7 - Create a sorted bar chart of Species Count with a Region Drop down selector

Create a bar chart with horizontal bars that shows the total species count per park with the color channel encoding the Region.

Add a drop down selector so that you can filter the bar chart by just one region. See [this documentation](https://altair-viz.github.io/gallery/multiple_interactions.html) for an example of adding a drop down menu and using it to filter your selections.

In addition to the drop down, include a `selection_multi` that will highlight the selected data by turning unselected data light gray.

In [20]:
# your code here

region_dropdown = alt.binding_select(options = [None, 'AK','IM','MW','NE','PW','SE'], labels = ['All','AK','IM','MW','NE','PW','SE'], name = 'Filter by a Region: ')

region_select = alt.selection_single(fields = ['Region'], bind = region_dropdown)

park_select = alt.selection_multi(fields = ['Park Name'])

bars = alt.Chart(df_species_final[df_species_final['Species Type'] == 'All Types']).mark_bar().encode(
    
    x = alt.X('Species Count:Q'),

    y = alt.Y('Park Name:N', sort = '-x'),

    color = alt.condition(park_select,
                          alt.Color('Region:O', 
                                    scale = alt.Scale(domain = ['AK','IM','MW','NE','PW','SE'],
                                                      range = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e', '#e6ab02', '#a6761d']
                                                      )
                          ),
                          alt.value('lightgray'),   
                          ),

    tooltip = ['Park Name', 'Species Count', 'Species Type']

).add_selection(
    
    park_select

).add_selection(
    
    region_select

).transform_filter(
    
    region_select,

).properties(title = 'Parks Sorted by Species Count')

bars

alt.Chart(...)

## Q8 - Bring all the charts together for a Dashboard

Bring all 4 charts and the region filter together.  Any click on one chart should highlight that data in the
other charts.  The colors for the regions should not change depending on the region selection and be consistent across 
all graphs. When plotting your geopandas data frame, filter out only the rows that have species data before making the map. I used the [documentation on customizing titles](https://altair-viz.github.io/user_guide/configuration.html#config-title) to change the font size and name my dashboard (all of the titles in the component graphs I changed to be `subtitles`)

Change your color scheme to `tableau10`.

Hint: There will be only one `selection_multi` that links all the selection highlighting in the charts together.

In [21]:
names = df_species_final['Park Name'].unique()

new_gdf = new_gdf[new_gdf['Park Name'].isin(names)]

In [22]:
# your code here

# The bars chart

region_dropdown = alt.binding_select(options = [None, 'AK','IM','MW','NE','PW','SE'], labels = ['All','AK','IM','MW','NE','PW','SE'], name = 'Filter by a Region: ')

region_select = alt.selection_single(fields = ['Region'], bind = region_dropdown)

park_select = alt.selection_multi(fields = ['Park Name'])

bars = alt.Chart(df_species_final[df_species_final['Species Type'] == 'All Types']).mark_bar().encode(
    
    x = alt.X('Species Count:Q'),

    y = alt.Y('Park Name:N', sort = '-x'),

    color = alt.condition(park_select,
                          alt.Color('Region:O', 
                                    scale = alt.Scale(domain = ['AK','IM','MW','NE','PW','SE'],
                                                      range = ['#5778a4', '#e49444', '#d1615d', '#85b6b2', '#6a9f58', '#e7ca60']
                                                      )
                          ),
                          alt.value('lightgray'),   
                          ),

    tooltip = ['Park Name', 'Species Count', 'Species Type']

).add_selection(
    
    park_select

).add_selection(
    
    region_select

).transform_filter(
    
    region_select,

).properties(title={
      "text" : ' ',
      "subtitle": ['Parks Sorted by Species Count']})

In [23]:
# your code here

# The map 

counties = alt.topo_feature(data.us_10m.url, 'counties')

basemap = alt.Chart(counties).mark_geoshape(
    
    opacity= 0.25,
    
    fill="lightgray", 
    
    stroke="darkgray",
    
).project(
    
    type = 'albersUsa'
    
).properties(
    
    width=1500, 
        
    height=500,
)

points = alt.Chart(new_gdf).mark_circle(size = 70).encode(
    
    longitude='geometry.coordinates[0]:Q', 

    latitude='geometry.coordinates[1]:Q',

    tooltip = ['Park Name', 'Region', 'Acres'],

    color = alt.condition(park_select,
                          alt.Color('Region:O', 
                                    scale = alt.Scale(domain = ['AK','IM','MW','NE','PW','SE'],
                                                      range = ['#5778a4', '#e49444', '#d1615d', '#85b6b2', '#6a9f58', '#e7ca60']
                                                      )
                          ),
                          alt.value('lightgray'),   
                          ),


).add_selection(
    
    park_select

).add_selection(
    
    region_select

).transform_filter(
    
    region_select
    
).properties(width=600, height=600, title={
      "text" : ' ',
      "subtitle": ['National Parks and Monuments in the USA']})

In [24]:
# your code here

# tick plot

plot = alt.Chart(df_species_final[df_species_final['Species Type'] != 'All Types']).mark_tick(thickness = 3).encode(
    
    x = alt.X('Species Count:Q'),

    y = alt.Y('Species Type:O', title = ''),

    color = alt.condition(park_select,
                          alt.Color('Region:O', 
                                    scale = alt.Scale(domain = ['AK','IM','MW','NE','PW','SE'],
                                                      range = ['#5778a4', '#e49444', '#d1615d', '#85b6b2', '#6a9f58', '#e7ca60']
                                                      )
                          ),
                          alt.value('lightgray'),   
                          ),

    size = alt.condition(park_select,
                         alt.value(20),
                         alt.value(15)
                         ),

    opacity = alt.condition(park_select,
                            alt.value(1),
                            alt.value(0.5),
                            ),
    
    tooltip = ['Park Name', 'Species Count', 'Species Type', 'Region'],

).add_selection(
    
    park_select

).add_selection(
    
    region_select

).transform_filter(
    
    region_select
    
).properties(width = 600, height = 500, title={
      "text" : ' ',
      "subtitle": ['Species Count by Type'] })

In [25]:
# your code here

scatter = alt.Chart(df_species_final[df_species_final['Species Type'] == 'All Types']).mark_circle(size = 70).encode(
    
    x = alt.X('Acres:Q',
              scale = alt.Scale(type = 'log')
              ),

    y = alt.Y('Species Count:Q',
              scale = alt.Scale(type = 'log')
              ),

    color = alt.condition(park_select,
                      alt.Color('Region:O', scale = alt.Scale(scheme='dark2')),
                      alt.value('lightgray')
                      ),
    tooltip = ['Park Name', 'Species Count', 'Acres']

).add_selection(
    
    park_select

).add_selection(
    
    region_select

).transform_filter(
    
    region_select
    
).properties( width = 600, height = 500, title={
      "text" : ' ',
      "subtitle": ['Park Size vs Specific Counts (log/log Scale)'] })

title = alt.Chart(
    {"values": [{"text": "National Park Diversity Dashboard"}]}
).mark_text(size=20).encode(
    text="text:N"
)

title & (bars | (basemap + points) & (plot | scatter))

alt.VConcatChart(...)